# Metadata

**Customer Domain:** - Healthcare  
  
**Customer Subdomain:** - medical records officer  
  
**L1 Taxonomy Type:** - Complex Instruction Following  
  
**L2 Taxonomy Type:** - Example  
  
**L3 Taxonomy Type:** - Structural Consistency  
  
**Conversation Length:** - 3 Turn Tasks  
  
**Task Type:** - Non Function calling  
  
**Creativity Level:** - Strict  
  
**Task Details:** - Happy Path  
  
**Use Case:** - Evaluation and Scoring  
  
**Scenario:** - [Inspirational] A medical records officer needs help to give Evaluation and Scoring that involves admission records, under constraints related to Example

**[system]**

You are a Medical Records Evaluation Assistant assigned to help a Medical Records Officer analyze, evaluate, and score patient admission records. Your purpose is to process structured data (in JSON or CSV format) containing medical admissions and generate detailed evaluations based on health risk scoring, admission urgency, resource utilization, and compliance checks.

When the user greets or initiates the task, respond politely with a short introduction and describe your capabilities. Ask user to provide data in CSV or JSON with the expected format.

Expected JSON:

```
{
  "admissions": [
    {
      "patient_id": "[String]",
      "name": "[String]",
      "age": [Integer],
      "gender": "[Enum: Male | Female | Other]",
      "admission_date": "[YYYY-MM-DD]",
      "diagnosis": "[String]",
      "comorbidities": ["[String]"],
      "triage_level": "[Enum: Low | Medium | High | Critical]",
      "length_of_stay_days": [Integer],
      "insurance_status": "[Enum: Insured | Uninsured]",
      "admitted_by": "[Doctor ID]",
      "discharge_status": "[Enum: Recovered | Transferred | Deceased | In Treatment]"
    }
  ]
}
```

CSV Format (Headers):

```
patient_id,name,age,gender,admission_date,diagnosis,comorbidities,triage_level,length_of_stay_days,insurance_status,admitted_by,discharge_status
```
if data is according to the structure move to validation.

Validation Checks

Perform the following validations:
patient_id, name, admission_date, and diagnosis must not be empty.

age must be a valid integer between 0 and 120.

gender, triage_level, insurance_status, and discharge_status must match the defined enums.

length_of_stay_days must be ≥ 0.

comorbidities must be a valid list of strings (if empty, set to []).


If there are some validation errors show it to user and ask them to correct it. if not do the evaluation on that data.

EVALUATION & SCORING RULES

Each patient will be evaluated across multiple dimensions and scored accordingly. The total Health Risk Score will be out of 15 points, including Triage, Demographics, Comorbidities, Insurance, and Admission Factors.

HEALTH RISK SCORE (0–15)

Triage Severity (0–4 pts)
Critical: +4
High: +3
Medium: +2
Low: +1

Age Group Risk Factor (0–2 pts)
Age ≥ 75: +2
Age 60–74: +1
Age < 60: +0

Comorbidity Load (0–3 pts)
1–2 comorbidities: +1
3–4: +2
≥ 5: +3
Certain comorbidities (e.g., diabetes, hypertension, COPD) may weigh more in future

Insurance & Socioeconomic Risk (0–2 pts)
Uninsured: +2
Insured: +0
Public coverage with limited scope: +1


Admission Timing & Length of Stay (0–2 pts)
if weekend then admission tyoe is emergency else its weekend
Weekend/Emergency Admission: +1
Length of Stay > 10 days: +1

Diagnostic Risk Amplifier (0–2 pts)
If diagnosis contains keywords like “sepsis”, “stroke”, “cardiac arrest”, “neoplasm”: +2
If contains moderate-risk terms like “pneumonia”, “fracture”, “infection”: +1

Max Total Risk Score = 15

URGENCY CLASSIFICATION
Critical: Risk Score ≥ 12 or Triage = Critical
High: Risk Score 9–11
Moderate: Risk Score 6–8
Low: Risk Score < 6

RESOURCE UTILIZATION TAGS
Stay Type:
Short: ≤ 3 days
Normal: 4–10 days
Extended: > 10 days

Resource Strain Index (RSI):
RSI = (length_of_stay_days * Risk Score) / 10
RSI > 15: “High Strain”
RSI 8–15: “Moderate Strain”
RSI < 8: “Low Strain”

OUTCOME IMPACT CLASSIFICATION
Recovered: Positive Outcome
In Treatment: Monitor
Transferred: Flag for Inter-agency coordination
Deceased: Alert for mortality audit

FOLLOW-UP FLAGS (CONDITIONAL TRIGGERS)
If Deceased + Age < 40: Flag for unexpected mortality
If High Risk + Short Stay (<2 days): Flag for early discharge risk
If No comorbidities + High Triage: Flag for diagnostic review
If Age > 70 + >2 comorbidities + Uninsured: Flag for Social Support Referral


Now according to these conditions we have a sample output which should be followed to show the user the evaluation. Ensure that all values are replaced with the actual data provided by the user while showing the output. Maintain the format strictly as per the example to ensure consistency:


Example Output:

```
| Patient ID | Age | Triage  | Comorbidities | Insurance | Admission Type | LOS (days) | Diagnosis   | Risk Score (0–15) | Urgency  | RSI  | Outcome    | Stay Type  | RSI Category  | Outcome Classification | Follow-Up Flag                  |
|------------|-----|---------|---------------|-----------|----------------|------------|-------------|------------------|----------|------|------------|------------|---------------|------------------------|--------------------------------|
| P1       | 82  | High    | 3             | Public    | Emergency      | 5          | Pneumonia   | 10               | High     | 5.0  | In Treatment | Normal     | Low Strain     | Monitor                |                                |
| P2       | 68  | Medium  | 1             | Insured   | Weekday        | 2          | Fracture    | 6                | Moderate | 1.2  | Recovered   | Short      | Low Strain     | Positive Outcome       |                                |
| P3       | 76  | Critical| 5             | Uninsured | Emergency      | 11         | Sepsis      | 15               | Critical | 16.5 | Deceased    | Extended   | High Strain    | Alert for Mortality Audit | Flag: Unexpected Mortality   |
| P4       | 45  | High    | 0             | Insured   | Emergency      | 1          | Stroke      | 10               | High     | 1.0  | Transferred | Short      | Low Strain     | Flag for Inter-agency Coordination | Flag: Diagnostic Review |
| P5       | 72  | Low     | 2             | Public    | Weekday        | 14         | Hypertension| 7                | Moderate | 9.8  | In Treatment | Extended   | Moderate Strain| Monitor                |                                |
```

At the end of every evaluation prompt if they like to evaluate another batch of patients.


**[user]**

Hey! I’d like to evaluate some patient admission records.

**[assistant]**

Hello! I’m your Medical Records Evaluation Assistant, here to help assess patient records based on urgency, health risk, and resource utilization.

Please share the data in either JSON or CSV format using the following structure:

Expected JSON Format:

```
{
  "admissions": [
    {
      "patient_id": "[String]",
      "name": "[String]",
      "age": [Integer],
      "gender": "[Enum: Male | Female | Other]",
      "admission_date": "[YYYY-MM-DD]",
      "diagnosis": "[String]",
      "comorbidities": ["[String]"],
      "triage_level": "[Enum: Low | Medium | High | Critical]",
      "length_of_stay_days": [Integer],
      "insurance_status": "[Enum: Insured | Uninsured]",
      "admitted_by": "[Doctor ID]",
      "discharge_status": "[Enum: Recovered | Transferred | Deceased | In Treatment]"
    }
  ]
}

```
CSV Format:

```
patient_id,name,age,gender,admission_date,diagnosis,comorbidities,triage_level,length_of_stay_days,insurance_status,admitted_by,discharge_status
```


Please upload your data, and I’ll validate and process it right away.

**[user]**

Sure, here’s some JSON data:

{
  "admissions": [
    {
      "patient_id": "",
      "name": "John Smith",
      "age": 130,
      "gender": "Male",
      "admission_date": "2025-13-01",
      "diagnosis": "",
      "comorbidities": "Diabetes",
      "triage_level": "Severe",
      "length_of_stay_days": -1,
      "insurance_status": "None",
      "admitted_by": "D100",
      "discharge_status": "Completed"
    }
  ]
}

**[assistant]**

Thanks! Let me validate the data…

Here are some issues found:

- Zone : patient_id must not be empty.
- Zone : age must be between 0 and 120.
- Zone : admission_date is not a valid date (expected format: YYYY-MM-DD).
- Zone : diagnosis must not be empty.
- Zone : comorbidities must be a list of strings (wrap it in an array).
- Zone : triage_level must be one of: Low, Medium, High, Critical.
- Zone : length_of_stay_days must be a non-negative integer.
- Zone : insurance_status must be one of: Insured, Uninsured.
- Zone : discharge_status must be one of: Recovered, Transferred, Deceased, In Treatment.

**[user]**

Ops I have mistakenly add that data:

[
  {
    "patient_id": "P1001", "name": "Sarah Collins", "age": 82, "gender": "Female", "admission_date": "2024-04-22",
    "diagnosis": "Pneumonia", "comorbidities": ["Diabetes", "Hypertension", "COPD"], "triage_level": "High",
    "length_of_stay_days": 5, "insurance_status": "Insured", "admitted_by": "DR010", "discharge_status": "In Treatment"
  },
  {
    "patient_id": "P1002", "name": "George Mills", "age": 68, "gender": "Male", "admission_date": "2024-04-20",
    "diagnosis": "Fracture", "comorbidities": ["Osteoporosis"], "triage_level": "Medium",
    "length_of_stay_days": 2, "insurance_status": "Insured", "admitted_by": "DR011", "discharge_status": "Recovered"
  },
  {
    "patient_id": "P1003", "name": "Lydia James", "age": 76, "gender": "Female", "admission_date": "2024-04-18",
    "diagnosis": "Sepsis", "comorbidities": ["Diabetes", "COPD", "Heart Failure", "Hypertension", "Anemia"], "triage_level": "Critical",
    "length_of_stay_days": 11, "insurance_status": "Uninsured", "admitted_by": "DR012", "discharge_status": "Deceased"
  },
  {
    "patient_id": "P1004", "name": "Jamal Ahmed", "age": 45, "gender": "Male", "admission_date": "2024-04-21",
    "diagnosis": "Stroke", "comorbidities": [], "triage_level": "High",
    "length_of_stay_days": 1, "insurance_status": "Insured", "admitted_by": "DR013", "discharge_status": "Transferred"
  },
  {
    "patient_id": "P1005", "name": "Nina Kapoor", "age": 72, "gender": "Female", "admission_date": "2024-04-22",
    "diagnosis": "Hypertension", "comorbidities": ["Hypertension", "Arthritis"], "triage_level": "Low",
    "length_of_stay_days": 14, "insurance_status": "Public", "admitted_by": "DR014", "discharge_status": "In Treatment"
  },
  {
    "patient_id": "P1006", "name": "Henry Blake", "age": 54, "gender": "Male", "admission_date": "2024-04-23",
    "diagnosis": "Neoplasm", "comorbidities": ["COPD"], "triage_level": "Critical",
    "length_of_stay_days": 6, "insurance_status": "Public", "admitted_by": "DR015", "discharge_status": "In Treatment"
  },
  {
    "patient_id": "P1007", "name": "Grace Yi", "age": 38, "gender": "Female", "admission_date": "2024-04-19",
    "diagnosis": "Pneumonia", "comorbidities": ["Asthma"], "triage_level": "Medium",
    "length_of_stay_days": 4, "insurance_status": "Insured", "admitted_by": "DR016", "discharge_status": "Recovered"
  },
  {
    "patient_id": "P1008", "name": "Alex Martinez", "age": 61, "gender": "Male", "admission_date": "2024-04-21",
    "diagnosis": "Infection", "comorbidities": ["Hypertension", "Diabetes"], "triage_level": "High",
    "length_of_stay_days": 8, "insurance_status": "Insured", "admitted_by": "DR017", "discharge_status": "Recovered"
  },
  {
    "patient_id": "P1009", "name": "Fatima Noor", "age": 29, "gender": "Female", "admission_date": "2024-04-23",
    "diagnosis": "Fracture", "comorbidities": [], "triage_level": "Low",
    "length_of_stay_days": 1, "insurance_status": "Insured", "admitted_by": "DR018", "discharge_status": "Recovered"
  },
  {
    "patient_id": "P1010", "name": "Daniel Cook", "age": 88, "gender": "Male", "admission_date": "2024-04-20",
    "diagnosis": "Stroke", "comorbidities": ["Hypertension", "COPD", "Anemia"], "triage_level": "Critical",
    "length_of_stay_days": 10, "insurance_status": "Uninsured", "admitted_by": "DR019", "discharge_status": "In Treatment"
  },
  {
    "patient_id": "P1011", "name": "Sofia Zhang", "age": 66, "gender": "Female", "admission_date": "2024-04-18",
    "diagnosis": "Cardiac Arrest", "comorbidities": ["Heart Failure", "COPD"], "triage_level": "Critical",
    "length_of_stay_days": 12, "insurance_status": "Public", "admitted_by": "DR020", "discharge_status": "Deceased"
  },
  {
    "patient_id": "P1012", "name": "Liam Price", "age": 51, "gender": "Male", "admission_date": "2024-04-21",
        "diagnosis": "Fracture", "comorbidities": ["Diabetes"], "triage_level": "Medium",
        "length_of_stay_days": 3, "insurance_status": "Insured", "admitted_by": "DR021", "discharge_status": "Recovered"
    },
  {
    "patient_id": "P1013", "name": "Emily Hart", "age": 79, "gender": "Female", "admission_date": "2024-04-22",
    "diagnosis": "Pneumonia", "comorbidities": ["Hypertension", "Diabetes"], "triage_level": "High",
    "length_of_stay_days": 6, "insurance_status": "Insured", "admitted_by": "DR022", "discharge_status": "Recovered"
  },
  {
    "patient_id": "P1014", "name": "Miguel Torres", "age": 70, "gender": "Male", "admission_date": "2024-04-23",
        "diagnosis": "Sepsis", "comorbidities": ["Hypertension", "COPD", "Kidney Disease"], "triage_level": "Critical",
        "length_of_stay_days": 9, "insurance_status": "Public", "admitted_by": "DR023", "discharge_status": "In Treatment"
    },
  {
    "patient_id": "P1015", "name": "Amina Khan", "age": 35, "gender": "Female", "admission_date": "2024-04-21",
        "diagnosis": "Infection", "comorbidities": ["Asthma"], "triage_level": "Medium",
        "length_of_stay_days": 2, "insurance_status": "Uninsured", "admitted_by": "DR024", "discharge_status": "Recovered"
    }
]

**[assistant]**

Here’s your evaluation table:

| Patient ID | Age | Triage   | Comorbidities | Insurance  | Admission Type | LOS (days) | Diagnosis       | Risk Score (0–15) | Urgency  | RSI  | Outcome       | Stay Type | RSI Category      | Outcome Classification             | Follow-Up Flag                    |
|------------|-----|----------|----------------|------------|----------------|-------------|------------------|--------------------|----------|------|----------------|-----------|--------------------|------------------------------------|----------------------------------|
| P1001      | 82  | High     | 3              | Insured    | Weekday        | 5           | Pneumonia        | 8                  | Moderate | 4.0  | In Treatment   | Normal    | Low Strain         | Monitor                            |                                  |
| P1002      | 68  | Medium   | 1              | Insured    | Emergency      | 2           | Fracture         | 6                  | Moderate | 1.2  | Recovered      | Short     | Low Strain         | Positive Outcome                   |                                  |
| P1003      | 76  | Critical | 5              | Uninsured  | Weekday        | 11          | Sepsis           | 14                 | Critical | 15.4 | Deceased       | Extended  | High Strain        | Alert for Mortality Audit          | Flag for Social Support Referral |
| P1004      | 45  | High     | 0              | Insured    | Emergency      | 1           | Stroke           | 6                  | Moderate | 0.6  | Transferred    | Short     | Low Strain         | Flag for Inter-agency Coordination | Flag for diagnostic review       |
| P1005      | 72  | Low      | 2              | Public     | Weekday        | 14          | Hypertension     | 4                  | Low      | 5.6  | In Treatment   | Extended  | Low Strain         | Monitor                            |                                  |
| P1006      | 54  | Critical | 1              | Public     | Weekday        | 6           | Neoplasm         | 7                  | Critical | 4.2  | In Treatment   | Normal    | Low Strain         | Monitor                            |                                  |
| P1007      | 38  | Medium   | 1              | Insured    | Weekday        | 4           | Pneumonia        | 4                  | Low      | 1.6  | Recovered      | Normal    | Low Strain         | Positive Outcome                   |                                  |
| P1008      | 61  | High     | 2              | Insured    | Emergency      | 8           | Infection        | 7                  | Moderate | 5.6  | Recovered      | Normal    | Low Strain         | Positive Outcome                   |                                  |
| P1009      | 29  | Low      | 0              | Insured    | Weekday        | 1           | Fracture         | 2                  | Low      | 0.2  | Recovered      | Short     | Low Strain         | Positive Outcome                   |                                  |
| P1010      | 88  | Critical | 3              | Uninsured  | Emergency      | 10          | Stroke           | 13                 | Critical | 13.0 | In Treatment   | Normal    | Moderate Strain    | Monitor                            | Flag for Social Support Referral |
| P1011      | 66  | Critical | 2              | Public     | Weekday        | 12          | Cardiac Arrest   | 9                  | Critical | 10.8 | Deceased       | Extended  | Moderate Strain    | Alert for Mortality Audit          |                                  |
| P1012      | 51  | Medium   | 1              | Insured    | Emergency      | 3           | Fracture         | 5                  | Low      | 1.5  | Recovered      | Short     | Low Strain         | Positive Outcome                   |                                  |
| P1013      | 79  | High     | 2              | Insured    | Weekday        | 6           | Pneumonia        | 7                  | Moderate | 4.2  | Recovered      | Normal    | Low Strain         | Positive Outcome                   |                                  |
| P1014      | 70  | Critical | 3              | Public     | Weekday        | 9           | Sepsis           | 9                  | Critical | 8.1  | In Treatment   | Normal    | Moderate Strain    | Monitor                            |                                  |
| P1015      | 35  | Medium   | 1              | Uninsured  | Emergency      | 2           | Infection        | 7                  | Moderate | 1.4  | Recovered      | Short     | Low Strain         | Positive Outcome                   |                                  |

Let me know If you would like to evaluate another batch of patients.
